In [ ]:
pip install dash dash_mantine_components dash_bootstrap_components pandas dash_iconify

In [158]:
from dash import Dash, dash_table, dcc, callback, Output, Input, clientside_callback, _dash_renderer
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_mantine_components as dmc
from requests import get
from dash.dependencies import MATCH, ALL, State
from dash.exceptions import PreventUpdate
from dash import callback_context
from dash import html
import webbrowser
from threading import Timer
from dash_iconify import DashIconify
import dash_bootstrap_components as dbc
import numpy as np
from pathlib import Path


In [159]:
DATA_DIR = Path("notebooks/dashboard")

# Mapping set ups to avoid retyping long string

In [ ]:

CSV_FILES = {
    "Category": DATA_DIR / "CPS_Summary.csv",
    "Brands per Category": DATA_DIR / "BPCPS_Summary.csv",
    "Products per Brand": DATA_DIR / "PPBPS_Summary.csv",
    "Products per Category": DATA_DIR / "PPCPS_Summary.csv",
}


In [161]:
type_map = {
        "Brands per Category": "brand",
        "Products per Category": "product",
        "Products per Brand": "product",
        "Category": "category"
    }

# Utils that are used in the combined update

In [162]:
def load_data(radio_choice):
    col = type_map[radio_choice]
    df = pd.read_csv(
        CSV_FILES[radio_choice],
        usecols=[col, 'year', 'month','Popularity Score']
    )
    return df

In [163]:
def sort_by_date(df, radio_choice):
        return df.sort_values(by='date', ascending=True)

In [164]:
def plot_bar_chart(filtered_df, selected_val, template, radio_choice):
    # print(filtered_df.head(3))
    col = type_map[radio_choice]
    if col == 'product':
        if radio_choice == 'Products per Brand':
            fig = px.bar(
                filtered_df,
                x="Popularity Score",
                y="product",
                color="product",   # 👈 change to 'month' or other column if you prefer
                orientation="h",
                template=template,
                title=f"Popularity of Products for {selected_val}"
            )

        if radio_choice == 'Products per Category':
            fig = px.bar(
                filtered_df,
                x="Popularity Score",
                y="product",
                color="product",   # 👈 change to 'month' or other column if you prefer
                orientation="h",
                template=template,
                title=f"Popularity of Products for {selected_val}"
        )

        fig.update_layout(
            xaxis_title="Popularity Score",
            yaxis_title="Product",
            margin=dict(l=200),
            legend=dict(
                orientation="h",    # horizontal
                x=0.5,              # center horizontally
                xanchor="center",
                y=-0.2,             # move legend below plot
                yanchor="top"
            )
        )

        fig.update_yaxes(showticklabels=False)

    else:
        fig = px.bar(sort_by_date(filtered_df, radio_choice), x="date", y='Popularity Score',
                    template=template, title=f"Top Categories in {selected_val}", color=col,
                    hover_name=col)
        
    return fig

In [165]:
def plot_hierarchy_chart(chart_type, template):
    df = pd.read_csv(CSV_FILES["Products per Brand"])
    if chart_type == 'sunburst':
        fig = px.sunburst(
            df,
            path = ['category', 'brand', 'product'],
            values='Popularity Score',
            color='category',
            template=template,
            title='Category -> Brand -> Product Hierarchy'
        )

    else: 
        fig = px.icicle(
            df,
            path=['category', 'brand', 'product'],
            values='Popularity Score',
            color='category',
            template=template,
            title="Category → Brand → Product Popularity (Icicle)"
        )

    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
    return fig

In [166]:
def plot_sankey_chart(df, template):
    categories = df['category'].unique().tolist()
    brands = df['brand'].unique().tolist()
    products = df['product'].unique().tolist()

    labels = categories + brands + products
    label_to_index = {label: i for i, label in enumerate(labels)}

    # Build Sankey links
    source, target, value = [], [], []

    # Category → Brand flows
    for _, row in df.groupby(['category', 'brand'])['Popularity Score'].sum().reset_index().iterrows():
        source.append(label_to_index[row['category']])
        target.append(label_to_index[row['brand']])
        value.append(row['Popularity Score'])

    # Brand → Product flows
    for _, row in df.groupby(['brand', 'product'])['Popularity Score'].sum().reset_index().iterrows():
        source.append(label_to_index[row['brand']])
        target.append(label_to_index[row['product']])
        value.append(row['Popularity Score'])

    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels
        ),
        link=dict(
            source=source,
            target=target,
            value=value
        )
    ))

    fig.update_layout(
        title_text="Flow: Category → Brand → Product Popularity",
        font_size=12,
        template=template
    )

    return fig

In [167]:
def plot_heatmap(df, template):
    heatmap_data = df.pivot_table(
        index = 'category',
        columns='brand',
        values='Popularity Score',
        aggfunc='sum',
        fill_value=0,
    )

    fig = px.imshow(
        heatmap_data,
        labels=dict(x='Brand', y='Category', color='Popularity Score'),
        x = heatmap_data.columns,
        y = heatmap_data.index,
        color_continuous_scale='Viridis',
        aspect='auto',
        template=template,
        title='Popularity Heatmap: Brand vs Categories',
        text_auto=False,
    )

    fig.update_traces(
        hovertemplate="Brand: %{x}<br>Category: %{y}<br>Popularity Score: %{z}<extra></extra>"
    )
    fig.update_xaxes(showticklabels=False)
    fig.update_layout(margin=dict(t=80, l=100, r=50, b=50))
    
    return fig

In [168]:
def plot_time_series(filtered_df, radio_choice, selected_val, template, compare):
    if not compare:
        fig = px.line(sort_by_date(filtered_df, radio_choice), x="date", y="Popularity Score",
                  template=template, title=f"{radio_choice} - {selected_val}")
    else:
        fig = px.line(sort_by_date(filtered_df, radio_choice), 
                      x='date', y='Popularity Score', color=type_map[radio_choice], template=template,
                      title=f"{radio_choice} Comparisons")
    
    return fig

In [169]:
def plot_treemap(filtered_df, radio_choice, template):
    if type_map[radio_choice] == 'category':
        fig = px.treemap(
            filtered_df, path=[px.Constant('all'), 'category'], 
            values='Popularity Score', title=f"Tree Map showing {radio_choice} popularity distribution",
            template=template,
        )

        fig.update_traces(root_color='grey')
        fig.update_layout(margin = dict(t=50, l=25, r=25, b=25),
                          width=1400,
                          height=800
                          )

    else:
        fig = px.treemap(
            filtered_df,
            path=[px.Constant('all'), 'category', 'brand'],
            values='Popularity Score',
            title=f"Treemap showing {radio_choice} popularity distribution (by Category → Brand)",
            template=template,
        )
        
        fig.update_traces(root_color='grey')
        fig.update_layout(margin=dict(t=50, l=25, r=25, b=25),
                          width=1600,
                          height=800
                          )

    return fig

Optional radar chart but will likely have to add a multiselect for category in order to use properly

In [170]:
def plot_radarchart(df, select_cat, multiselect, template):
    categories = [select_cat]

    fig = go.Figure()

    for brand in df['brand'].unique():

        fig.add_trace(
            go.Scatterpolar(
            r = df['Popularity Score'],
            theta = categories,
            fill = 'toself',
            name = brand,
            )
        )

    fig.update_layout(
        polar = dict(
            radialaxis = dict(
                visible = True,
                range=[0, df.max()]
            )
        )
    )

In [171]:
def plot_groupedbar(df, radio_choice, template):
    if radio_choice == "Brands per Category":
        fig = px.bar(
            df,
            x="brand",
            y="Popularity Score",
            color="category",          # categories grouped by color
            barmode="group",
            template=template,
            title=f"{radio_choice} - Brands grouped by Category",
            hover_name="brand",        # <-- shows brand in hover tooltip
            hover_data={"category": True, "Popularity Score": True}
        )

    elif radio_choice == "Products per Category": 
        fig = px.bar(
            df,
            x="product",
            y="Popularity Score",
            color="category",          # categories grouped by color
            barmode="group",
            template=template,
            title=f"{radio_choice} - Products grouped by Category",
            hover_name="product",        # <-- shows brand in hover tooltip
            hover_data={"product": True, "Popularity Score": True}
        )

    else: 
        return []

    fig.update_xaxes(showticklabels=False)

    fig.update_traces(width=0.8)

    fig.update_layout(
        bargap=0.1,        # space between groups (default ~0.2)
        bargroupgap=0.05   # space between bars within a group (default ~0.1)
    )

    return fig

In [172]:
def get_template(theme):
    dmc.add_figure_templates()
    return "mantine_dark" if theme == "dark" else "mantine_light"

In [173]:
def get_table_styles(theme):
    if theme == "dark":
        return {
            "header": {
                "backgroundColor": "#1f2937",
                "color": "white",
                "fontWeight": "bold",
                "textAlign": "center"
            },
            "data": {
                "backgroundColor": "#111827",
                "color": "white",
                "border": "1px solid #374151"
            },
            "data_conditional": [
                {
                    "if": {"row_index": "odd"},
                    "backgroundColor": "#1f2937"
                }
            ]
        }
    else:  # light theme
        return {
            "header": {
                "backgroundColor": "#f3f4f6",
                "color": "#111827",
                "fontWeight": "bold",
                "textAlign": "center"
            },
            "data": {
                "backgroundColor": "white",
                "color": "#111827",
                "border": "1px solid #d1d5db"
            },
            "data_conditional": [
                {
                    "if": {"row_index": "odd"},
                    "backgroundColor": "#f9fafb"
                }
            ]
        }

In [174]:
def get_data_table(df, theme):
    styles = get_table_styles(theme)

    df = df.copy()
    # df['date'] = df['date'].dt.strftime('%Y-%m-%d')  # nicer date format

    table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        style_table={"overflowX": "auto"},
        style_cell={"padding": "8px", "textAlign": "left", "fontSize": "14px"},
        style_header=styles['header'],
        style_data=styles['data'],
        style_data_conditional=styles['data_conditional'],
    )

    return table

In [175]:
theme_toggle = dmc.Switch(
    offLabel=DashIconify(icon="radix-icons:sun", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][8]),
    onLabel=DashIconify(icon="radix-icons:moon", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][6]),
    id="color-scheme-switch",
    persistence=True,
    color="grey",
)

In [176]:
def load_prod_data(radio_choice, select_cat, select_brand, template, theme, compare, multiselect):
    
    df = load_data(radio_choice)
    grouped_bar = []

    # if not compare:
    if radio_choice == "Products per Category":
        category = pd.read_csv(CSV_FILES[radio_choice], usecols=['category'])
        df['category'] = category['category']

        # select_year = select_brand
        options = [{'label': val, 'value': val} for val in sorted(df['category'].unique())]

        if not select_cat:
            return(
                options, 
                html.Div('Select a category to view product performance'),
                {'display':'none'},
                [],
                {'display':'none'},
                [],
                {'display':'none'},
                [],
                {'display':'block'},
                [],
                [],
                [],
                []
            )
        
        else:

            filtered_df = df[df['category']==select_cat]
            top_prod = filtered_df.head(10)

        top_prod['month_year'] = top_prod['month'].astype(str) + "-" + top_prod['year'].astype(str)
        top_prod['date'] = pd.to_datetime(top_prod[["year", "month"]].assign(day=1))

        time_series = dcc.Graph(figure=plot_time_series(top_prod,radio_choice,select_cat,template, compare))
        barchart = dcc.Graph(figure=plot_bar_chart(top_prod, select_cat, template, radio_choice))
        grouped_bar = dcc.Graph(figure=plot_groupedbar(top_prod, radio_choice, template))

        return(
            options, 
            time_series, 
            {'display': 'none'}, 
            [],
            {'display': 'none'},
            get_data_table(top_prod, theme),
            {'display':'none'},
            [],
            {'display':'block'},
            [],
            grouped_bar,
            barchart,
            [],
        )
        

            
    else:
        brand = pd.read_csv(CSV_FILES[radio_choice], usecols=['brand'])
        df['brand'] = brand['brand']
        # print(df.head(3))

        # select_year = select_brand
        options = [{'label': val, 'value': val} for val in sorted(df['brand'].unique()[:750])]

        if not select_cat:
            return(
                options, 
                html.Div('Select a brand to view product performance'),
                {'display':'none'},
                [],
                {'display':'none'},
                [],
                {'display':'none'},
                [],
                {'display':'block'},
                [],
                [],
                [],
                []
            )
        
        else:
            filtered_df = df[df['brand']==select_cat]
            top_prod = filtered_df.head(10)
        
        top_prod['month_year'] = top_prod['month'].astype(str) + "-" + top_prod['year'].astype(str)
        top_prod['date'] = pd.to_datetime(top_prod[["year", "month"]].assign(day=1))

        time_series = dcc.Graph(figure=plot_time_series(top_prod,radio_choice,select_cat,template, compare))
        barchart = dcc.Graph(figure=plot_bar_chart(top_prod, select_cat, template, radio_choice))

        return(
            options, 
            time_series, 
            {'display': 'none'}, 
            [],
            {'display': 'none'},
            get_data_table(top_prod, theme),
            {'display':'none'},
            [],
            {'display':'block'},
            [],
            [],
            barchart,
            [],
        )

In [177]:
def load_brand_data(radio_choice, select_cat, select_brand, template, theme, compare, multiselect):

    brand_df = pd.read_csv(CSV_FILES[radio_choice])
        
    cat_options = [{'label': val, 'value': val} for val in sorted(brand_df['category'].unique())]

    dfs = [
        brand_df[brand_df['category'] == opt['value']]
        .drop_duplicates(subset="brand")
        .head(3)
        for opt in cat_options
    ]
    
    filtered_data = pd.concat(dfs, ignore_index=True)

    heatmap = dcc.Graph(figure=plot_heatmap(filtered_data, template))

    grouped_bar = dcc.Graph(figure=plot_groupedbar(filtered_data, radio_choice, template))
    treemap = dcc.Graph(figure=plot_treemap(filtered_data, radio_choice, template))

    brand_df = brand_df[brand_df['category'] == select_cat].copy()

    brand_options = [{'label': val, 'value': val} for val in brand_df['brand'].unique()]

    if not compare:
        if not select_cat:
            return(
                cat_options, 
                html.Div('Select a category to view brand performance'),
                {'display': 'block'},
                [],
                {'display': 'block'},
                [],
                {'display': 'none'},
                [],
                {'display':'block'},
                treemap,
                grouped_bar,
                [],
                heatmap,
            )

        if not select_brand:
            return(
                cat_options,
                html.Div('Select a brand to view performance'),
                {'display': 'block'},
                brand_options,
                {'display': 'block'},
                [],
                {'display': 'none'},
                [],
                {'display':'block'},
                treemap,
                grouped_bar,
                [],
                heatmap,
            )
        
        
        filtered_df = brand_df[brand_df['brand'] == select_brand].copy()
        filtered_df['date'] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

        time_series = dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, select_cat, template, compare))

        return(
            cat_options,
            time_series,
            {'display': 'block'}, 
            brand_options,
            {'display': 'block'},
            get_data_table(filtered_df, theme),
            {'display': 'none'},
            [],
            {'display':'block'},
            treemap,
            grouped_bar,
            [],
            heatmap,
        )


    else:
        if not select_cat:
            return(
                cat_options, 
                html.Div('Select a category to view brand performance'),
                {'display': 'block'},
                [],
                {'display': 'none'},
                [],
                {'display': 'block'},
                [],
                {'display':'block'},
                treemap,
                grouped_bar,
                [],
                heatmap,
            )
        
        if not multiselect:
            return(
                cat_options,
                html.Div('Select a brand to compare performance'),
                {'display': 'block'},
                [],
                {'display': 'none'},
                [],
                {'display':'block'},
                brand_options,
                {'display':'block'},
                treemap,
                grouped_bar,
                [],
                heatmap,
            )
        
        dfs = [brand_df[brand_df['brand'] == select] for select in multiselect]
        for df in dfs:
            df['date'] = pd.to_datetime(df[["year", "month"]].assign(day=1))

        filtered_df = pd.concat(dfs, ignore_index=True)

        time_series = dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, select_cat, template, compare))
        return(
            cat_options,
            time_series,
            {'display': 'block'},
            [],
            {'display': 'none'},
            [],
            {'display':'block'},
            brand_options,
            {'display':'block'},
            treemap,
            grouped_bar,
            [],
            heatmap,
        )

In [178]:
def load_cat_data(radio_choice, select_cat, select_brand, template, theme, compare, multiselect):

    df = load_data(radio_choice)
    options = [{'label': val, 'value': val} for val in sorted(df['category'].unique())]
    treemap = dcc.Graph(figure=plot_treemap(df, radio_choice, template))
    df['date'] = pd.to_datetime(df[["year", "month"]].assign(day=1))
    barchart = dcc.Graph(figure=plot_bar_chart(df, select_cat, template, radio_choice))
    # sunburst = dcc.Graph(figure=plot_hierarchy_chart('sunburst', template))

    table = []

    if not compare:
        
        if not select_cat:
            return(
                options, 
                html.Div('Select a category to view brand performance'),
                {'display': 'block'},
                [],
                {'display': 'none'},
                [],
                {'display':'none'},
                [],
                {'display':'block'},
                treemap,
                [],
                barchart,
                []
            )
        
        
        filtered_df = df[df['category'] == select_cat].copy()
        filtered_df['date'] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

        
        table = get_data_table(filtered_df, theme)
        time_series = dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, select_cat, template, compare))

        return(
            options, 
            time_series, 
            {'display': 'block'},
            select_brand,
            {'display': 'none'},
            table,
            {'display':'none'},
            [],
            {'display':'block'},
            treemap,
            [],
            barchart,
            []
        )

    else:

        if not multiselect:
            return(
                [],
                html.Div('Select categories to compare performance'),
                {'display': 'block'},
                [],
                {'display': 'none'},
                [],
                {'display':'block'},
                options,
                {'display':'none'},
                treemap,
                [],
                barchart,
                []
            )

        dfs = [df[df['category'] == select] for select in multiselect]
        for df in dfs:
            df['date'] = pd.to_datetime(df[["year", "month"]].assign(day=1))

        filtered_df = pd.concat(dfs, ignore_index=True)

        time_series = dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, select_cat, template, compare))

        return(
            [],
            time_series,
            {'display': 'block'},
            [],
            {'display': 'none'},
            table,
            {'display':'block'},
            options,
            {'display':'none'},
            treemap,
            [],
            barchart,
            []
        )

# Actual app layout and combined update

In [179]:
app = Dash()

app.layout = dmc.MantineProvider(
    children=[
        theme_toggle,
        dcc.Store(id='theme-store'),
        dmc.Container([

            dmc.Title("Market Analysis Dashboard", 
                      style={"size":"h3", "textAlign": "center"}), 
        
            dmc.Grid([
                dmc.GridCol(
                    dmc.RadioGroup(
                    [dmc.Radio(i, value=i) for i in CSV_FILES.keys()],
                    id="radio-group",
                    value="Category",
                    size="md",
                    style={"marginBottom": "25px"}
                    ),
                    span=8
                ),

                dmc.GridCol(
                    dmc.Checkbox(
                    id="compare-checkbox",
                    labelPosition="right",
                    label="Compare",
                    color="#5c7cfa",
                    variant="filled",
                    size="sm",
                    radius="sm",
                    disabled=False,
                    indeterminate=False,
                    style={'display': 'block', 'justifyContent': 'center',
                        'margonBottom': '25px'},
                    ),
                    span=4,
                    style={"display": "flex", 
                           "alignItems": "top", 
                           "justifyContent": "flex-end"}
                ),
            ]),
            
            dmc.Select(
                id="first-dropdown",
                data=[],
                placeholder="Select a category to view specific info",
                searchable=True,
                clearable=True,
                nothingFoundMessage="No matches found",
            ),

            dmc.Space(h=20),

            dmc.Select(
                id='second-dropdown',
                data=[],
                placeholder="Select a brand to view specific info",
                searchable=True,
                clearable=True,
                nothingFoundMessage="No matches found",
                style={'display': 'none'},
            ),

            dmc.MultiSelect(
                placeholder='Select multiple to compare',
                id='multiselect-dropdown',
                data=[],
                searchable=True,
                clearable=True,
                nothingFoundMessage="No matches found",
                style={'display':'none'},
            ),

            dmc.Space(h=10),

            html.Div(id='treemap'),

            dmc.Space(h=10),

            html.Div(id='barchart'),

            dmc.Space(h=10),

            html.Div(id='heatmap'),

            dmc.Space(h=10),

            html.Div(id='grouped-bar'),

            dmc.Space(h=10),

            html.Div(id="time-series"),
            
            dmc.Space(h=10),
            
            html.Div(id='data-table'),

            dmc.Space(h=20)

        ], fluid=True,
    )],
)
@callback(
    Output("first-dropdown", "data"),
    Output("time-series", "children"),
    Output("compare-checkbox", "style"),
    Output("second-dropdown", "data"),
    Output('second-dropdown', 'style'),
    Output('data-table', 'children'),
    Output('multiselect-dropdown', 'style'),
    Output('multiselect-dropdown','data'),
    Output('first-dropdown', 'style'),
    Output('treemap', 'children'),
    Output('grouped-bar', 'children'),
    Output('barchart', 'children'),
    Output('heatmap', 'children'),
    
    Input("radio-group", "value"),
    Input("first-dropdown", "value"),
    Input("theme-store", "data"),
    Input('second-dropdown', 'value'),
    Input('compare-checkbox', 'checked'),
    Input('multiselect-dropdown','value'),
)
def combined_update(radio_choice, select_cat, theme, select_brand, compare, multiselect):
    template = get_template(theme)
    
    col_map = {
        "Brands per Category": load_brand_data,
        "Products per Category": load_prod_data,
        "Products per Brand": load_prod_data,
        "Category": load_cat_data
    }

    if radio_choice in col_map:
        return col_map[radio_choice](radio_choice, select_cat, select_brand, template, theme, compare, multiselect)
    
    return (
        [], 
        html.Div("Invalid selection."), 
        {'display': 'none'}, 
        [], 
        {'display':'none'}, 
        [], 
        {'display':'none'}, 
        [], 
        {'display':'none'}, 
        [], 
        [],
        [],
        [],
        [],
    )
    
clientside_callback(
    """
    (switchOn) => {
       const theme = switchOn ? 'dark' : 'light';
        document.documentElement.setAttribute('data-mantine-color-scheme', theme);
        return theme;
    }
    """,
    Output("theme-store", "data"),
    Input("color-scheme-switch", "checked"),
)


def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

if __name__ == "__main__":
    Timer(1, open_browser).start()  # Wait 1 second then open browser
    app.run(debug=True)